In [4]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains
import csv
import numpy as np
import pandas as pd
import requests
import re

In [48]:
# provide PATH variable dependent on browser user commands Selenium to use, below is the commented code for chrome
# PATH = "C:\Program Files (x86)\chromedriver.exe"
def sign_in(email, password, PATH):
#     this function automates login process on linkedin, 
#     provide email and password as strings
    driver = webdriver.Chrome(PATH)
    time.sleep(5)
    # webdriver gets page
    driver.get("https://www.linkedin.com/")
    # pause for page to load
    time.sleep(3)
    # locate and send login email and password
    driver.find_element_by_id("session_key").send_keys(email)
    driver.find_element_by_id("session_password").send_keys(password)
    driver.find_element_by_class_name("sign-in-form__submit-button").click()
    return driver

In [150]:
def people_scrape(search_term, num_pages):
           # create lists to hold data  
    loc_list = []
    current_job_list = []
    name_list = []
    url_list = []
    headline_list = []
#                 function to automate search bar, search focus
#                 from your linkedin homepage, collecting data from
#                 search including name, location(secondary_deets), 
#                 headline(primary_deets) from condensed profiles returned from
#                 search results. requires string entry for search term
#                 and int input for number of pages (num_pages) 
#                 that you wish to scrape from results.

    # activate search bar cursor with click
    driver.find_element_by_css_selector("div#global-nav-search ").click()
    time.sleep(2)
    # send keyboard entry "div[id='oc-background-section']")for search terms
    driver.find_element_by_css_selector("input.search-global-typeahead__input").send_keys(search_term)
    # send enter key to activate search
    driver.find_element_by_css_selector("input.search-global-typeahead__input").send_keys(Keys.RETURN)
    # wait for results to load
    driver.implicitly_wait(6)
    #w.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'ul.reusable-search__entity-results-list')))
    element = driver.find_element_by_css_selector('ul.reusable-search__entity-results-list ')
    # scroll to element containing target(people_banner)  allowing ajax elements to load
    driver.execute_script("arguments[0].scrollIntoView();", element)
    driver.implicitly_wait(5)
   # locate banner under search type results
    people_banner = driver.find_element_by_link_text(f"See all people results")
    # js function to click banner/button to see additional results under jobs, people, or posts                      
    driver.execute_script('arguments[0].click();',people_banner)
    # pause to allow page to load
    driver.implicitly_wait(6)
    # starting with pagination page 1                       
    page_number = 1
    # get current url for page
    current_page_url = driver.current_url

    # set while loop to define pagination and data collection conditions
    while page_number <= num_pages:
        print("Processing page: " + str(page_number))
    # find all results on page
        links = driver.find_elements_by_css_selector("div.entity-result__content ")
   # pause for page load
        time.sleep(2)
    
    # iterate through results
        for l in links:
       # retrieve profile url
            title = l.find_element_by_css_selector("span.entity-result__title a.app-aware-link")
            profile_path = (str(title.get_attribute("pathname")))
    # add to urls list
            url_list.append(profile_path) 
        
    # locating elements containing text needed
            details = l.find_elements_by_css_selector("div.linked-area")
    # the first element has the first three lines of text in the container
            deets = details[0]  
    # split text to assign elements appropriately
            text = deets.get_attribute('innerText').split('\n')
    # retrieve name/add to list
            name = text[0]
            name_list.append(name)
            
    # retrieve location/add to list   
            loc = text[-1]
            loc_list.append(loc)
            
     # retrieve headline/add to list
            headline = text[-2]
            headline_list.append(headline)
            
#             if len(details) >= 1:
#     # the second element selected contains the 'Current:' job text 
#                 current_job = details[1]
#     # removing the 'Current:' string from text
#                 current_job = current_job.get_attribute('innerText').split(':')[1]
#             else:
#                 current_job = 'nan'
#     # add current job to list
#             current_job_list.append(current_job)
        time.sleep(3)
    # navigate using pagination function
        goto_next_page()
        page_number+=1
    # print to verify page during processing
        print(f"attempting to navigate to search results page {page_number}")
        time.sleep(5)
   
    # create dataframe with extracted information and save as csv file
    df = pd.DataFrame()                      
    df['name'] = name_list
    df['url'] = url_list
    #df['current_job'] = current_job_list
    df['location'] = loc_list
    df['headline'] = headline_list
    # add complete url information for use in complete profile scraping
    for row in df:
        df['fetch'] = 'https://www.linkedin.com' + df.url + '/'
    df.to_csv(f'{search_term}.csv')
    # verify save
    print(f'{search_term}.csv saved')
    return df

In [147]:
def manual_search_scrape(num_pages, save_label):
    loc_list = []
    current_job_list = []
    name_list = []
    url_list = []
    headline_list = []
        
#                 function used when user inputs search manually, collecting data from
#                 search including name, location(secondary_deets), 
#                 headline(primary_deets) from condensed profiles returned from
#                 search results. requires string entry for search term
#                 and int input for number of pages (num_pages) 
#                 that you wish to scrape from results.


    # starting with pagination page 1                       
    page_number = 1

    # set while loop to define pagination and data collection conditions
    while page_number <= num_pages:
    # find all results on page
        links = driver.find_elements_by_css_selector("div.entity-result__content ")
   # pause for page load
        time.sleep(2)
    
    # iterate through results
        for l in links:
    # retrieve profile url
            title = l.find_element_by_css_selector("span.entity-result__title a.app-aware-link")
            profile_path = (str(title.get_attribute("pathname")))
    # add to urls list
            url_list.append(profile_path)
            print(profile_path)
    # locating elements containing text needed
            details = l.find_elements_by_css_selector("div.linked-area")
    # the first element has the first three lines of text in the container
            deets = details[0]  
    # split text to assign elements appropriately
            text = deets.get_attribute('innerText').split('\n')
    # retrieve name/add to list
            name = text[0]
            name_list.append(name)
            print(name)
    # retrieve location/add to list   
            loc = text[-1]
            loc_list.append(loc)
     # retrieve headline/add to list
            headline = text[-2]
            headline_list.append(headline)
    # the second element selected contains the 'Current:' job text 
            current_job = details[1]
    # removing the 'Current:' string from text
            current_job = current_job.get_attribute('innerText').split(':')[1]
    # add current job to list
            current_job_list.append(current_job)
        time.sleep(3)
        print("Processed page: " + str(page_number))
    # navigate using pagination function
        goto_next_page()
        page_number+=1
    # print to verify page during processing
        print(f"attempting to navigate to search results page {page_number}")
        time.sleep(5)
   
    # create dataframe with extracted information and save as csv file
    df = pd.DataFrame()                      
    df['name'] = name_list
    df['url'] = url_list
    #df['current_job'] = current_job_list
    df['location'] = loc_list
    df['headline'] = headline_list
    # add complete url information for use in complete profile scraping
    for row in df:
        df['fetch'] = 'https://www.linkedin.com' + df.url + '/'
    df.to_csv(f'{save_label}.csv')
    # verify save
    print(f'{save_label}.csv saved')
    return df

In [97]:
#function to locate and interact with "next" button at bottom of search
def goto_next_page():
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(3)
    driver.find_element_by_css_selector('div.artdeco-pagination')
    go_to_next = driver.find_element_by_css_selector('button[aria-label="Next"]')
    driver.execute_script('arguments[0].click();',go_to_next)

In [119]:
# def get_experience():
#     w = WebDriverWait(driver, 10)
#     w.until(EC.presence_of_element_located((By.CSS_SELECTOR,"div.profile-detail")))
#     time.sleep(5)
#     # background container
#     background = driver.find_element_by_css_selector(
#         "div[id='oc-background-section']")
#     # scroll element into view enabling location of elements within for dynamic loading page
#     driver.execute_script("arguments[0].scrollIntoView();", background)
#     time.sleep(8)
#     # try/except clause used to locate element, not all people list jobs
#     try:
#         exp = background.find_element_by_css_selector(
#                 "section#experience-section.pv-profile-section.experience-section")
    
#     except (NoSuchElementException, IndexError):
#         pos = 'nan'
#         company= 'nan'
#         location = 'nan'
#         dates='nan'
#     history = exp.find_elements_by_css_selector('li.pv-entity__position-group-pager')
#     details = history[0]
#     # try/except clause used to locate element to avoid error ceasing from running
#     try:
#         pos = details.find_element_by_tag_name(
#                 'h3').get_attribute('outerText')
#     except NoSuchElementException:
#         pos = 'nan'

#     try:
#         company = details.find_element_by_tag_name(
#                 'p.pv-entity__secondary-title').get_attribute('innerText')
        
#     except NoSuchElementException:
#         company = 'nan'
#     try:
#         location = details.find_element_by_css_selector(
#                 'h4.pv-entity__location').get_attribute('innerText')
#         location = location.split('\n', 1)[1]
#     except NoSuchElementException:
#         location = 'nan'
        
#     try:
#         dates = details.find_element_by_css_selector(
#                 "h4.pv-entity__date-range").get_attribute('innerText').split(' ', 2)[-1]

#     except NoSuchElementException:
#             dates = 'nan'
#     finally:
#         job_list.append(pos)
#         company_list.append(company)
#         date_list.append(dates)
#         loc_list.append(location)

#     if (len(history)) >=2:
#         details = history[1]
        
#         # job title
#         try:
#             pos = details.find_element_by_tag_name(
#                     'h3').get_attribute('outerText')
#         except NoSuchElementException:
#             pos = 'nan'
            
#         # company of employment
#         try:
#             company = details.find_element_by_tag_name(
#                     'p.pv-entity__secondary-title').get_attribute('innerText')
#         except NoSuchElementException:
#             company = 'nan'
            
#         # location of employment
#         try:
#             location = details.find_element_by_css_selector(
#                     'h4.pv-entity__location').get_attribute('innerText')
#             location = location.split('\n', 1)[1]
#         except NoSuchElementException:
#             location = 'nan'
            
#        # dates of employment
#         try:
#             dates = details.find_element_by_css_selector(
#                     "h4.pv-entity__date-range").get_attribute('innerText').split(' ', 2)[-1]

#         except NoSuchElementException:
#                 dates = 'nan'

#         if (len(history)) < 2:
#             pos = 'nan'
#             company  ='nan'
#             location  ='nan'
#             dates = 'nan'

#         job_list2.append(pos)
#         company_list2.append(company)
#         date_list2.append(dates)
#         loc_list2.append(location)
#     driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")    
    
#     time.sleep(2)



In [99]:
def get_experience():
    job_list = []
    company_list = []
    location_list = []
    date_list = []
#     job_list2 = []
#     company_list2 = []
#     loc_list2 = []
#     date_list2 = []
    time.sleep(3)
    w = WebDriverWait(driver, 10)
    w.until(EC.presence_of_element_located((By.CSS_SELECTOR,"h2.pv-profile-section__card-heading")))
    h2_heading = driver.find_element_by_css_selector(
        "h2.pv-profile-section__card-heading")
    driver.execute_script("arguments[0].scrollIntoView();", h2_heading)
    time.sleep(3)
    # background container
    background = driver.find_element_by_css_selector(
        "div[id='oc-background-section']")
    # scroll background container into view to access elements within
    driver.execute_script("arguments[0].scrollIntoView();", background)
    # wait for section to be accessible
    w.until(EC.presence_of_element_located((By.CSS_SELECTOR,"section#experience-section")))
    # locate element and access it
    test = background.find_elements_by_css_selector(
                "section#experience-section.pv-profile-section.experience-section")
    if test >=1:
        exp = background.find_element_by_css_selector(
                "section#experience-section.pv-profile-section.experience-section")

        history = exp.find_elements_by_css_selector('li.pv-entity__position-group-pager')
        num_jobs = len(history)
        job_count_list.append(num_jobs)
        details = history[0]
        # try/except clause used to locate element to avoid 'element not found' error which halts program
        # job title
        try:
            pos = details.find_element_by_tag_name(
                        'h3').get_attribute('innerText')
        except NoSuchElementException:
            pos = 'nan'

            # company of employment
        try:
            company = details.find_element_by_tag_name(
                        'p.pv-entity__secondary-title').get_attribute('innerText')
        except NoSuchElementException:
            company = 'nan'

            # location of employment
        try:
            location = details.find_element_by_css_selector(
                        'h4.pv-entity__location').get_attribute('innerText')
            location = location.split('\n', 1)[1]
        except NoSuchElementException:
            location = 'nan'

           # dates of employment
        try:
            dates = details.find_element_by_css_selector(
                        "h4.pv-entity__date-range").get_attribute('innerText').split(' ', 2)[-1]

        except NoSuchElementException:
                    dates = 'nan'
    else:
        pos = 'nan'
        company= 'nan'
        location = 'nan'
        dates='nan'
        # save to lists to create dataframe        
    job_list.append(pos)
    company_list.append(company)
    date_list.append(dates)
    location_list.append(location)

    #     # for second job data retrieval
    #     if (len(history)) >=2:
    #         details = history[1]

#         # job title
#         try:
#             pos = details.find_element_by_tag_name(
#                     'h3').get_attribute('outerText')
#         except NoSuchElementException:
#             pos = 'nan'
            
#         # company of employment
#         try:
#             company = details.find_element_by_tag_name(
#                     'p.pv-entity__secondary-title').get_attribute('innerText')
#         except NoSuchElementException:
#             company = 'nan'
            
#         # location of employment
#         try:
#             location = details.find_element_by_css_selector(
#                     'h4.pv-entity__location').get_attribute('innerText')
#             location = location.split('\n', 1)[1]
#         except NoSuchElementException:
#             location = 'nan'
            
#        # dates of employment
#         try:
#             dates = details.find_element_by_css_selector(
#                     "h4.pv-entity__date-range").get_attribute('innerText').split(' ', 2)[-1]

#         except NoSuchElementException:
#                 dates = 'nan'

#         if (len(history)) < 2:
#             pos = 'nan'
#             company  ='nan'
#             location  ='nan'
#             dates = 'nan'

#         job_list2.append(pos)
#         company_list2.append(company)
#         date_list2.append(dates)
#         loc_list2.append(location)
        
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")    
    time.sleep(2)
    return job_list, company_list, date_list, location_list, job_count_list

In [100]:
def get_education():
    school_list = []
        # function accessing and saves profile school information, optional second job retrieval commented out below
    # wait for element to proceed
    w = WebDriverWait(driver, 10)
    w.until(EC.presence_of_element_located((By.CSS_SELECTOR,"div.profile-detail")))
    # background container
    try:
        background = driver.find_element_by_css_selector(
            "div[id='oc-background-section']")
    # scroll into view to access elements within
        driver.execute_script("arguments[0].scrollIntoView();", background)
        time.sleep(8)
    # locate education section
        education = background.find_element_by_css_selector(
            "section[id='education-section']")
        schools = education.find_elements_by_css_selector(
                "a[data-control-name='background_details_school']")
        school = schools[0]
        school_name = school.find_element_by_tag_name(
            'h3').get_attribute('innerText')
    except NoSuchElementException:
        school_name = 'nan'

    school_list.append(school_name)
    return school_list

In [101]:
def get_email():
#     function accessing and retrieves email and name from profile header, extracts first name 
    names_list = []
    fname_list = []
    email_list = []
    # scroll to top of profile where email is located
    driver.execute_script("window.scrollTo(0, 0);")
    time.sleep(2)
    header = driver.find_elements_by_css_selector(
        "ul.pv-top-card--list")
    name = header[0].get_attribute('innerText')
    names_list.append(name)
    fname = name.split(' ')
    first_name = name[0]
    fname_list.append(first_name)
    contact_info = driver.find_element_by_css_selector(
        'a[data-control-name="contact_see_more"]')
     # js function to click banner/button to see additional results under jobs, people, or posts                      
    driver.execute_script('arguments[0].click();',contact_info)
    time.sleep(3)
    try:
        container =  driver.find_elements_by_css_selector(
            'div.pv-contact-info__ci-container')
        email =  container[1].get_attribute('innerText')
     
    
    except (NoSuchElementException, IndexError):
        email = 'nan'
    
    email_list.append(email)
    close_popup = driver.find_element_by_css_selector(
            'button[aria-label="Dismiss" ]')
    close_popup.click()
    return names_list, fname_list, email_list

In [102]:
def profile_connect(message):
    # function used to interact with profile and send personalized message and connection request
    # scroll to top of profile to ensure elements can be found by webdriver
    driver.execute_script("window.scrollTo(0, 0);")
    time.sleep(5)
    # locate & click 'Connect' button located at top of profile
    driver.find_element_by_class_name('pv-s-profile-actions--connect').click()
    time.sleep(4)
    action_bar = driver.find_element_by_css_selector("div.artdeco-modal__actionbar")
    # locate 'Add a note' button by class
    action_bar.find_element_by_class_name('mr1').click()
    time.sleep(3)
    # action to send text from message to input box
    message_input = driver.find_element_by_id('custom-message').send_keys(message)
    # send the message & connection request
    message_input.find_element_by_id('custom-message').send_keys(Keys.RETURN)
    # send the message & connection request
    # driver.find_element_by_class_name('ml1').click()

In [103]:
def profile_scrape(profile_urls):
    for url in profile_urls:
        driver.get(url)
        get_experience()
        get_education()
        get_email()
    df = pd.DataFrame()
    df['first_name'] = fname_list
    df['email'] = email_list
    df['profile_name'] = names_list
    df['job1'] = job_list
    df['company1'] = company_list
    df['location1'] = loc_list
    df['dates1'] = date_list
    df['number of jobs'] = job_count_list
#     df['job2'] = job_list2
#     df['company2'] = company_list2
#     df['location2'] = loc_list2
#     df['dates2'] = date_list2
#     df['school'] = school_list
    return df

In [104]:
def make_connection(url_message_dict):
    counter = 1

    for key, value in url_message_dict.items():
        url = key
        message = value
        driver.get(url)
        profile_connect(message)
        print('connection to : ' +url+ ' is complete')
        counter+=1
    print(counter+' connections successful')

In [105]:
def search_scrape_connect(search_term, num_pages):

    search_df = people_scrape(search_term, num_pages)

    profile_urls = search_df['fetch']
   
    detail_df = profile_scrape(profile_urls)

    df = pd.concat([search_df, detail_df], axis=1)
    df = df.loc[df.company1 != 'nan']
    df = df.loc[df.job1 != 'nan']
  
    # each user to create message string with format options
    df['personalized_message'] = ("Hi " + df.first_name +", my name is Brennan, and I am a data scientist/ml engineer in the DMV area."
                                                       "My background is in video editing and QA. I see that you work as a " + df.job1 + "at "+ df.company1+
                                                       ", which sounds super interesting, so I just wanted to reach out, connect, and say hello!")  
    message_inputs = list(df['personalized_message'])
    urls = list(df['fetch'])
    connection_dict = dict(zip(urls, message_inputs))
    df.to_csv(search_term+'_comp.csv')
    make_connection(connection_dict)
    return df
    #driver.quit()